In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 
import sqlite3
import math
torch.cuda.is_available()

False

In [17]:
# Read in the data from the database
conn = sqlite3.connect('data/tables.db')
# show database content
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
# Extract two tables from it and store them in two pd df
ds = pd.read_sql_query("SELECT * from df_2023_h1_feature", conn)
target = pd.read_sql_query("SELECT * from df_2023_h1_target", conn)

[('df_2023_h1_feature',), ('df_2023_h1_target',)]


In [47]:
# custom train test val split

X_train = ds[0:10000]
y_train = target[0:10000]
X_val = ds[10001:11001]
y_val = target[10001:11001]
X_test = ds[11002: 12002]
y_test = target[11002: 12002]
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(10000, 22) (10000, 1) (1000, 22) (1000, 1) (1000, 22) (1000, 1)


In [48]:
train_data = X_train.join(y_train)
val_data = X_val.join(y_val)

In [55]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label='discounted_price').fit(train_data, val_data, hyperparameters='light', presets='medium_quality')
# predictor = TabularPredictor(label='discounted_price').fit(train_data, hyperparameters='light', presets='high_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240305_002020"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240305_002020"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.3.0: Wed Dec 20 21:33:31 PST 2023; root:xnu-10002.81.5~7/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       1.38 GB / 8.00 GB (17.2%)
Disk Space Avail:   207.70 GB / 460.43 GB (45.1%)
Train Data Rows:    10000
Train Data Columns: 22
Tuning Data Rows:    1000
Tuning Data Columns: 22
Label Column:       discounted_price
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (323.9548593148663, -220.80113456717004, 1.69532, 73.60656)
	If 'regres

[1000]	valid_set's rmse: 4.7472
[2000]	valid_set's rmse: 4.57188
[3000]	valid_set's rmse: 4.49188
[4000]	valid_set's rmse: 4.43771
[5000]	valid_set's rmse: 4.39204
[6000]	valid_set's rmse: 4.36758
[7000]	valid_set's rmse: 4.35015
[8000]	valid_set's rmse: 4.32531
[9000]	valid_set's rmse: 4.31542
[10000]	valid_set's rmse: 4.30819


	-4.3074	 = Validation score   (-root_mean_squared_error)
	29.23s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 1.29278
[2000]	valid_set's rmse: 1.27079
[3000]	valid_set's rmse: 1.2601
[4000]	valid_set's rmse: 1.25681
[5000]	valid_set's rmse: 1.25384
[6000]	valid_set's rmse: 1.25317
[7000]	valid_set's rmse: 1.25308
[8000]	valid_set's rmse: 1.25269
[9000]	valid_set's rmse: 1.25251
[10000]	valid_set's rmse: 1.25237


	-1.2523	 = Validation score   (-root_mean_squared_error)
	29.08s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-3.0092	 = Validation score   (-root_mean_squared_error)
	4.31s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost ...
	-2.2835	 = Validation score   (-root_mean_squared_error)
	39.8s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-3.5477	 = Validation score   (-root_mean_squared_error)
	0.82s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-2.0073	 = Validation score   (-root_mean_squared_error)
	3.94s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	-1.5231	 = Validation score   (-root_mean_squared_error)
	6.22s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-5.2274	 = Validation score   (-root_mean_squared_error)
	2.23s	 = Training   runtime
	0.01s	 = Validation run

[1000]	valid_set's rmse: 1.69672
[2000]	valid_set's rmse: 1.69267


	-1.6924	 = Validation score   (-root_mean_squared_error)
	36.24s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBM': 0.377, 'NeuralNetFastAI': 0.361, 'CatBoost': 0.164, 'XGBoost': 0.066, 'LightGBMXT': 0.033}
	-0.8949	 = Validation score   (-root_mean_squared_error)
	0.07s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 153.63s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240305_002020")


In [56]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.894902,root_mean_squared_error,0.474811,108.326183,0.000162,0.067670,2,True,10
1,LightGBM,-1.252302,root_mean_squared_error,0.230433,29.076741,0.230433,29.076741,1,True,2
2,XGBoost,-1.523076,root_mean_squared_error,0.022918,6.215938,0.022918,6.215938,1,True,7
3,LightGBMLarge,-1.692418,root_mean_squared_error,0.046503,36.241249,0.046503,36.241249,1,True,9
4,NeuralNetFastAI,-2.007261,root_mean_squared_error,0.010409,3.936366,0.010409,3.936366,1,True,6
5,CatBoost,-2.283518,root_mean_squared_error,0.006205,39.795120,0.006205,39.795120,1,True,4
6,RandomForestMSE,-3.009246,root_mean_squared_error,0.027005,4.314750,0.027005,4.314750,1,True,3
7,ExtraTreesMSE,-3.547691,root_mean_squared_error,0.048711,0.815917,0.048711,0.815917,1,True,5
8,LightGBMXT,-4.307437,root_mean_squared_error,0.204684,29.234348,0.204684,29.234348,1,True,1
9,NeuralNetTorch,-5.227427,root_mean_squared_error,0.006333,2.228752,0.006333,2.228752,1,True,8


In [57]:
from sklearn import metrics

In [61]:
y_pred = predictor.predict(X_test)
print(metrics.r2_score(y_true=y_test, y_pred=y_pred))
print(metrics.mean_squared_error(y_true=y_test, y_pred=y_pred))

0.9905235642099366
136.58152828031177


In [62]:
# try test data that is further away
X_test = ds[100000:101000]
y_test = target[100000:101000]
y_pred = predictor.predict(X_test)
print("Predictions:  \n", y_pred)
print(metrics.r2_score(y_true=y_test, y_pred=y_pred))
print(metrics.mean_squared_error(y_true=y_test, y_pred=y_pred))

Predictions:  
 100000   -92.247437
100001   -91.822105
100002   -91.558266
100003   -91.231705
100004   -90.913010
            ...    
100995   -41.729061
100996   -37.296516
100997   -33.999031
100998   -30.852152
100999   -27.195103
Name: discounted_price, Length: 1000, dtype: float32
0.7815977101398484
3627.590307650451
